In [96]:
# Read the csv dataset
import py_entitymatching as em
A = em.read_csv_metadata('../datasets/yelp_data.csv')
A['ID'] = range(0, len(A))
em.set_key(A, 'ID')

B = em.read_csv_metadata('../datasets/inspection_data.csv')
B['ID'] = range(0, len(B))
em.set_key(B, 'ID')



Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


True

In [97]:
A.head()

,name,address,zipcode,cuisine,ID
0,barn joo 35,"34 w 35th st, ,",10001,korean gastropubs tapas/small plates,0
1,friedman's,"132 w 31st st, ,",10001,american (new) gluten-free breakfast & brunch,1
2,juniper,"237 w 35th st,",10001,cocktail bars american (new),2
3,haymaker bar and kitchen,"252 w 29th st, ,",10001,gastropubs beer bar,3
4,izakaya mew,"53 w 35th st, basement,",10001,sushi bars izakaya cocktail bars,4


In [98]:
# Down-sample
sample_A, sample_B = em.down_sample(A, B, size=5000, y_param=2)

0%                          100%
[##############################] | ETA: 00:00:07 | ETA: 00:00:07 | ETA: 00:00:06 | ETA: 00:00:06 | ETA: 00:00:06 | ETA: 00:00:05 | ETA: 00:00:05 | ETA: 00:00:05 | ETA: 00:00:05 | ETA: 00:00:04 | ETA: 00:00:04 | ETA: 00:00:04 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:07


In [99]:
# Perform blocking
ab = em.AttrEquivalenceBlocker()
C = ab.block_tables(sample_A, sample_B, 'zipcode', 'zipcode', l_output_attrs=['name', 'address', 'zipcode', 'cuisine'], r_output_attrs=['name', 'address', 'zipcode', 'cuisine'])
ob = em.OverlapBlocker()
D = ob.block_candset(C, 'address', 'address', overlap_size=3)

0%                          100%
[##############################] | ETA: 00:00:04 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:04


In [100]:
# Describe the candidate set
print(len(D))
D.head(50)

2568


,_id,ltable_ID,rtable_ID,ltable_name,ltable_address,ltable_zipcode,ltable_cuisine,rtable_name,rtable_address,rtable_zipcode,rtable_cuisine
105,105,2,450,juniper,"237 w 35th st,",10001,cocktail bars american (new),dunkin donuts,155 w 35th st,10001,donuts
111,111,2,6988,juniper,"237 w 35th st,",10001,cocktail bars american (new),wafels & dinges,102 w 35th st,10001,french
131,131,2,4900,juniper,"237 w 35th st,",10001,cocktail bars american (new),sushi fussion,224 w 35th st,10001,jewish/kosher
140,140,2,12576,juniper,"237 w 35th st,",10001,cocktail bars american (new),sushi fussion,224 w 35th st,10001,jewish/kosher
163,163,2,17149,juniper,"237 w 35th st,",10001,cocktail bars american (new),dunkin donuts,155 w 35th st,10001,donuts
266,266,10,11526,jongro bbq,"22 w 32nd st, ,",10001,korean barbeque beer bar,shanghai mong,30 w 32nd st,10001,korean
267,267,10,3646,jongro bbq,"22 w 32nd st, ,",10001,korean barbeque beer bar,maru,11 w 32nd st,10001,korean
285,285,10,16410,jongro bbq,"22 w 32nd st, ,",10001,korean barbeque beer bar,miss korea barbecue ''sun'',10 w 32nd st,10001,korean
311,311,10,3661,jongro bbq,"22 w 32nd st, ,",10001,korean barbeque beer bar,paik's noodle,2 w 32nd st,10001,korean
323,323,10,21402,jongro bbq,"22 w 32nd st, ,",10001,korean barbeque beer bar,food gallery 32,11 w 32nd st,10001,korean
